In [1]:
!pip install datasets evaluate sentencepiece sacrebleu pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requir

In [3]:
import os
import json
import time
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
import re
import evaluate

# Set environment variable for API key (example, remove in production)
# Create your PAT token by following instructions here:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# grab the token from the token.txt file
with open("token.txt", "r") as f:
  GITHUB_TOKEN = f.read().strip()


# Initialize the OpenAI client (replace base_url if needed)
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=api_key=GITHUB_TOKEN
)

# Define models
MODEL_CONFIG = [
    ("gpt-4o-mini", "ChatGPT"),
    ("Codestral-2501", "Codestral"),
]

model_choices, model_names = zip(*MODEL_CONFIG)

# Updated EXAMPLES dictionary with all 22 tasks in chronological order
EXAMPLES = {
    "task01_code_summarization": {
        "type": "analysis",
        "snippet": """public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}
""",
        "strategies": {
            "zero_shot": "Summarize the functionality of the following Java method:",
            "few_shot": "Example:\nInput: a Java method that counts the frequency of each character in a string.\nOutput: This method takes a string as input and returns a map where each key is a character from the string, and the value is the number of times that character appears.\n\nNow, summarize the functionality of the following Java method:"
        }
    },
    "task02_bug_fixing_off_by_one": {
        "type": "analysis",
        "snippet": """def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
""",
        "strategies": {
            "few_shot": "Example:\nInput: start=1, end=4 → Output: 6 (This sums 1+2+3)\nNow identify and fix the off-by-one error in the following function:",
            "chain_of_thought": "Think step-by-step to find and correct the off-by-one error in the following Python function:"
        }
    },
    "task03_bug_classification_cpp": {
        "type": "analysis",
        "snippet": """int * getArray(int size) {
    int arr[size]; // Warning: local array
    return arr;    // Bug: returning pointer to local variable
}
""",
        "strategies": {
            "chain_of_thought": "Walk through your reasoning step-by-step and classify the bug in the following C++ function:",
            "prompt_chaining": "Step 1: Identify the bug in the following C++ function.\nStep 2: Explain why it occurs and its consequences."
        }
    },
    "task04_email_validator": {
        "type": "analysis",
        "snippet": """def is_valid_email(email):
    # TODO: Complete using regex
    pass
""",
        "strategies": {
            "prompt_chaining": "Step 1: Provide a regex pattern for basic email addresses (e.g., user@domain.com).\nStep 2: Implement `is_valid_email` using that pattern in the following code:",
            "chain_of_thought": "Think step-by-step to derive a regex and implement `is_valid_email` in the following code:"
        }
    },
    "task05_flask_api": {
        "type": "generation",
        "expected": """from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    return jsonify({'greeting': f'Hello, {username}!'})
""",
        "strategies": {
            "zero_shot": "Create a Flask `/greet/<username>` endpoint that returns a JSON greeting.",
            "self_consistency": "Create a Flask `/greet/<username>` endpoint that returns a JSON greeting."
        }
    },
    "task06_sql_schema": {
        "type": "generation",
        "expected": """-- users(id INT PRIMARY KEY, name VARCHAR(100));
-- books(id INT PRIMARY KEY, title VARCHAR(200));
-- reviews(id INT PRIMARY KEY, user_id INT REFERENCES users(id), book_id INT REFERENCES books(id), rating INT);
""",
        "strategies": {
            "zero_shot": "Write SQL schema (tables, primary & foreign keys) for a review app with users, books, and reviews. -- TODO : Design schema with appropriate keys and constraints -- Tables : users (id , name ), books (id , title ), reviews (id , user_id , book_id , rating )",
            "self_consistency": "Write SQL schema (tables, primary & foreign keys) for a review app with users, books, and reviews. -- TODO : Design schema with appropriate keys and constraints -- Tables : users (id , name ), books (id , title ), reviews (id , user_id , book_id , rating )"
        }
    },
    "task07_null_deref_java": {
        "type": "analysis",
        "snippet": """public int getLength(String s) {
    return s.length(); // What if s is null?
}
""",
        "strategies": {
            "few_shot": "Example:\nInput: a method with a null check before length().\nOutput: Checks if the string is null before calling length to avoid null-dereference.\nNow identify null-dereference risk in the following code:",
            "chain_of_thought": "Reason step-by-step to find any null-dereference risk in the following Java code:"
        }
    },
    "task08_csv_parser": {
        "type": "analysis",
        "snippet": """def parse_csv_line(line):
    return line.split(',')  # Incomplete: doesn't handle quoted fields
""",
        "strategies": {
            "few_shot": "Example:\nInput: '\"a,b\",c' → Output: ['a,b','c']\nNow improve the following CSV parser:",
            "chain_of_thought": "Think step-by-step to split CSV fields correctly, handling quotes, in the following code:"
        }
    },
    "task09_kotlin_api": {
        "type": "analysis",
        "snippet": """data class Product(val id: Int, val name: String, val price: Double)
// TODO: Create GET and POST endpoints using Ktor
""",
        "strategies": {
            "prompt_chaining": "Step 1: Define GET `/product/{id}` returning JSON.\nStep 2: Define POST endpoint accepting a Product in the body for the following code:",
            "self_consistency": "Convert the following Kotlin data class to a REST API with GET and POST endpoints using Ktor."
        }
    },
    "task10_func_summary_py": {
        "type": "analysis",
        "snippet": """def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
""",
        "strategies": {
            "prompt_chaining": "Step 1: Describe what the following function does.\nStep 2: Summarize it in one sentence:",
            "self_consistency": "Write a brief summary of the following function."
        }
    },
    "task11_prompt_from_comments": {
        "type": "analysis",
        "snippet": """# This function checks if a number is prime
def is_prime(n):
    if n <= 1: return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0: return False
    return True
""",
        "strategies": {
            "chain_of_thought": "Explain step-by-step how to craft a prompt to generate the following code:",
            "prompt_chaining": "Step 1: Write a system instruction that sets the task.\nStep 2: Provide the comment and ask for implementation for the following code:"
        }
    },
    "task12_fix_factorial_bug": {
    "type": "bug_fixing",
    "snippet": "def factorial(n):\n    result = 1\n    for i in range(1, n):\n        result *= i\n    return result",
    "strategies": {
        "self-consistency": "The logic fails for n = 0 due to incorrect loop bounds. Revise it so the function returns 1 for 0 and behaves consistently for all inputs.",
        "chain-of-thought": "Step-by-step, identify how the loop behaves for n = 0 and adjust it to include the base case explicitly."
    }
},
"task13_delete_linked_list_node": {
    "type": "data_structure",
    "snippet": "struct Node {\n    int data;\n    struct Node *next;\n};\nvoid deleteNode(struct Node **head, int key) {\n    // TODO : Implement node deletion\n}",
    "strategies": {
        "few-shot": "Example:\nInput: struct Node *head; int key = 4;\nOutput: Node with value 4 is removed from the list.\n\nNow remove the node with the given key from the linked list.",
        "chain-of-thought": "Step-by-step, walk through identifying the target node and safely unlinking it."
    }
},
"task14_recursive_fibonacci": {
    "type": "function_completion",
    "snippet": "def fibonacci(n):\n    # TODO : Base cases and recursive call\n    pass",
    "strategies": {
        "zero-shot": "Complete the recursive Fibonacci function to return the nth number.",
        "few-shot": "Example:\nInput: fibonacci(0) → 0\nInput: fibonacci(1) → 1\nOutput: fibonacci(5) → 5\n\nNow implement the recursive function:"
    }
},
"task15_constructor_completion": {
    "type": "class_definition",
    "snippet": "class Person:\n    def __init__(self):\n        # TODO : Add name , age , and optional email\n        pass",
    "strategies": {
        "zero-shot": "Write a constructor that initializes `name`, `age`, and optional `email` fields.",
        "prompt chaining": "Step 1: Identify which attributes should be stored.\nStep 2: Use parameters in `__init__` to initialize them."
    }
},
"task16_binary_search_java": {
    "type": "algorithm_completion",
    "snippet": "public int binarySearch(int[] arr, int target) {\n    int left = 0, right = arr.length - 1;\n    while (left <= right) {\n        int mid = (left + right) / 2;\n        // TODO : Compare and adjust bounds\n    }\n    return -1;\n}",
    "strategies": {
        "few-shot": "Example:\nInput: binarySearch([1, 3, 5, 7], 5) → 2\nNow implement binary search to find the target in the array.",
        "chain-of-thought": "Step-by-step, adjust the left/right bounds depending on how mid compares to the target."
    }
},
"task17_fix_even_logic": {
    "type": "self_consistency",
    "snippet": "bool isOdd(int x) {\n    return x % 2 == 0;\n}",
    "strategies": {
        "self-consistency": "The function is named isOdd but returns true for even numbers. Fix the logic or rename the function to match its behavior.",
        "chain-of-thought": "Step-by-step, identify the contradiction between the function’s intent and logic, then revise accordingly."
    }
},
"task18_fix_js_is_even": {
    "type": "bug_fixing",
    "snippet": "function isEven(n) {\n    return n % 2; // Returns 1 or 0 , not true / false\n}",
    "strategies": {
        "zero-shot": "Fix the function so it returns a Boolean value when checking for even numbers.",
        "prompt chaining": "Step 1: Identify what `n % 2` returns.\nStep 2: Modify the function so the result is explicitly a Boolean."
    }
},
"task19_decompose_summary_cpp": {
    "type": "code_decomposition",
    "snippet": "int process(int x) {\n    if (x < 0) return -1;\n    return x * x;\n}",
    "strategies": {
        "chain-of-thought": "Step-by-step, explain what the function checks and how it transforms the input.",
        "prompt chaining": "Step 1: Validate input.\nStep 2: Describe the square operation and why it's used."
    }
},
"task20_calculate_average": {
    "type": "function_completion",
    "snippet": "def calculate_average(scores):\n    total = 0\n    # TODO : Complete to return average\n    pass",
    "strategies": {
        "few-shot": "Example:\nInput: [90, 80, 70] → Output: 80.0\nNow complete the function to return the average score.",
        "chain-of-thought": "Step-by-step, explain how to sum the elements and divide by the count for the average."
    }
},
"task21_refactor_utils": {
    "type": "refactoring",
    "snippet": "# utils.py\nimport csv\n\ndef read_csv(filepath):\n    with open(filepath, 'r') as f:\n        return [row for row in csv.reader(f)]\n\ndef summarize_column(data, index):\n    values = [float(row[index]) for row in data[1:]]\n    total = sum(values)\n    avg = total / len(values)\n    return total, avg\n\ndef main():\n    filepath = 'data.csv'\n    data = read_csv(filepath)\n    total, avg = summarize_column(data, 1)\n    print(\"Total:\", total)\n    print(\"Average:\", avg)\n\nif __name__ == '__main__':\n    main()",
    "strategies": {
        "prompt chaining": "Step 1: Read the CSV file safely.\nStep 2: Summarize the column.\nStep 3: Identify areas to improve structure or error handling.",
        "self-consistency": "Ensure the logic is robust when parsing input and summarizing data. Refactor to avoid assumptions like non-empty input."
    }
},
"task22_complete_file_processor": {
    "type": "function_completion",
    "snippet": "# file_processor.py\nimport string\n\ndef load_file(filepath):\n    with open(filepath, 'r') as f:\n        return f.readlines()\n\ndef clean_line(line):\n    # TODO : Remove punctuation and make lowercase\n    pass\n\ndef count_words(lines):\n    word_counts = {}\n    for line in lines:\n        clean = clean_line(line)\n        for word in clean.split():\n            word_counts[word] = word_counts.get(word, 0) + 1\n    return word_counts\n\ndef main():\n    filepath = 'input.txt'\n    lines = load_file(filepath)\n    counts = count_words(lines)\n    for word, count in sorted(counts.items()):\n        print(f\"{word}: {count}\")\n\nif __name__ == '__main__':\n    main()",
    "strategies": {
        "zero-shot": "Complete the missing cleaning logic and make sure word frequencies are calculated correctly.",
        "prompt chaining": "Step 1: Describe how to normalize and clean a line of text.\nStep 2: Integrate the cleaned lines into the word counting logic."
    }
}

}


# Helper to build the prompt based on task type
def build_prompt(prefix, snippet=None):
    if snippet:
        lang = 'java' if 'public' in snippet else 'python' if 'def' in snippet else 'cpp'
        prompt = f"{prefix}\n```{lang}\n{snippet}\n```"
    else:
        prompt = f"{prefix}\n```{snippet}\n```"
    print("📝 PROMPT:\n", prompt)    # ← right here
    return prompt

# Safe chat completion with retry
def safe_chat_completion(model, prompt, max_retries=3):
    for attempt in range(1, max_retries + 1):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1024,
                temperature=0.7
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            msg = str(e)
            m = re.search(r"wait (\d+) seconds", msg)
            if m:
                wait_secs = int(m.group(1)) + 1
                print(f"🔒 Rate-limit on '{model}', sleeping {wait_secs}s then retry (attempt {attempt})...")
                time.sleep(wait_secs)
                continue
            backoff = 2 ** attempt
            print(f"⚠️  Error on attempt {attempt} for '{model}': {e}")
            print(f"  → backing off {backoff}s before retry.")
            time.sleep(backoff)
    print(f"❌ All retries failed for model '{model}'.")
    return None

# Self-consistency runner using BLEU-4 similarity among outputs
def run_self_consistency(model, prompt, runs=3):
    outputs = []
    for i in range(1, runs + 1):
        print(f"  • SC run {i}/{runs} for model '{model}'...")
        out = safe_chat_completion(model, prompt)
        if out:
            outputs.append(out)
    if not outputs:
        return None, []
    if len(outputs) == 1:
        return outputs[0], outputs

    # Load BLEU metric
    bleu = evaluate.load("sacrebleu")

    # Compute average BLEU-4 score of each output against others
    avg_scores = []
    for i, cand in enumerate(outputs):
        score_sum = 0.0
        count = 0
        for j, ref in enumerate(outputs):
            if i == j:
                continue
            result = bleu.compute(predictions=[cand], references=[[ref]])
            score_sum += result["score"]
            count += 1
        avg_scores.append(score_sum / count if count else 0.0)

    # Select the output with the highest average BLEU score
    best_idx = avg_scores.index(max(avg_scores))
    return outputs[best_idx], outputs

# Worker executed in parallel
def worker(args):
    task_key, strat_key, model_id, model_name, prompt = args
    print(f"→ Starting: {task_key} | {strat_key} @ {model_name}")
    timestamp = datetime.utcnow().isoformat()
    if "self_consistency" in strat_key:
        output, variants = run_self_consistency(model_id, prompt)
    else:
        output = safe_chat_completion(model_id, prompt)
        variants = []
    print(f"✓ Completed: {task_key} | {strat_key} @ {model_name}")
    return {
        "task": task_key,
        "strategy": strat_key,
        "model": model_name,
        "model_id": model_id,
        "prompt": prompt,
        "output": output,
        "variants": variants,
        "timestamp": timestamp
    }

# Orchestrator: builds jobs, runs them, and saves outputs
def generate_all():
    jobs = []
    for task_key, info in EXAMPLES.items():
        snippet = info.get("snippet")
        for strat_key, prefix in info["strategies"].items():
            prompt = build_prompt(prefix, snippet)
            for model_id, model_name in MODEL_CONFIG:
                jobs.append((task_key, strat_key, model_id, model_name, prompt))
    print(f"🔎 Total jobs to run: {len(jobs)}")
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(worker, jobs))
    df = pd.DataFrame(results)
    df.to_csv("all_prompts_and_outputs.csv", index=False)
    df.to_json("all_prompts_and_outputs.json", orient="records", indent=2)
    print("✅ All outputs have been generated and saved.")

# Run the full pipeline
if __name__ == '__main__':
    generate_all()


📝 PROMPT:
 Summarize the functionality of the following Java method:
```java
public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}

```
📝 PROMPT:
 Example:
Input: a Java method that counts the frequency of each character in a string.
Output: This method takes a string as input and returns a map where each key is a character from the string, and the value is the number of times that character appears.

Now, summarize the functionality of the following Java method:
```java
public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}

```
📝 PROMPT:
 Example:
Input: start=1, end=4 → Output: 6 (This sums 1+2+3)
Now identify an

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  • SC run 2/3 for model 'Codestral-2501'...


✓ Completed: task05_flask_api | self_consistency @ Codestral
→ Starting: task07_null_deref_java | few_shot @ ChatGPT
✓ Completed: task05_flask_api | self_consistency @ ChatGPT
→ Starting: task07_null_deref_java | few_shot @ Codestral
✓ Completed: task07_null_deref_java | few_shot @ ChatGPT
→ Starting: task07_null_deref_java | chain_of_thought @ ChatGPT
  • SC run 2/3 for model 'gpt-4o-mini'...
✓ Completed: task07_null_deref_java | few_shot @ Codestral
→ Starting: task07_null_deref_java | chain_of_thought @ Codestral
  • SC run 3/3 for model 'Codestral-2501'...
  • SC run 3/3 for model 'gpt-4o-mini'...
✓ Completed: task07_null_deref_java | chain_of_thought @ Codestral
→ Starting: task08_csv_parser | few_shot @ ChatGPT
✓ Completed: task07_null_deref_java | chain_of_thought @ ChatGPT
→ Starting: task08_csv_parser | few_shot @ Codestral
✓ Completed: task08_csv_parser | few_shot @ Codestral
→ Starting: task08_csv_parser | chain_of_thought @ ChatGPT
✓ Completed: task06_sql_schema | self_cons

In [ ]:
from google.colab import drive
drive.mount('/content/drive')